In [47]:
# Cell 1: Import tất cả các thư viện cần thiết
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from torch.amp import autocast 

In [48]:
# Cell 2: Kiểm tra phiên bản PyTorch
print(f"PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"CUDA available: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA not available, using CPU")

PyTorch version: 2.6.0+cu118
CUDA available: NVIDIA GeForce RTX 3050 Laptop GPU


In [49]:
# Cell 2: Định nghĩa lớp Dataset (tái sử dụng từ dataset_dataloader.ipynb)
class M5MultiDataset(Dataset):
    def __init__(self, data, feature_cols, window_size):
        self.data = data
        self.cols = feature_cols
        self.win = window_size

    def __len__(self):
        return len(self.data) - self.win

    def __getitem__(self, idx):
        slice_ = self.data.iloc[idx:idx+self.win]
        X_num = slice_[self.cols].values.astype('float32')  # (win, num_features)
        item_idx = slice_['item_idx'].values.astype('int64')  # (win,)
        store_idx = slice_['store_idx'].values.astype('int64')  # (win,)
        y = self.data['sales'].iloc[idx+self.win].astype('float32')
        return (
            torch.tensor(X_num),  # (win, num_features)
            torch.tensor(item_idx),  # (win,)
            torch.tensor(store_idx)  # (win,)
        ), torch.tensor(y)

In [50]:
# Cell 3: Định nghĩa lớp mô hình LSTMEmbForecast
class LSTMEmbForecast(nn.Module):
    def __init__(self, n_items, n_stores, embed_dim, num_feats, hidden_size):
        super().__init__()
        self.item_emb = nn.Embedding(n_items, embed_dim)
        self.store_emb = nn.Embedding(n_stores, embed_dim)
        self.lstm = nn.LSTM(
            input_size=num_feats + 2*embed_dim,
            hidden_size=hidden_size,
            num_layers=2,
            batch_first=True,
            dropout=0.2
        )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, X_num, X_item, X_store):
        batch_size, seq_len, _ = X_num.size()
        emb_i = self.item_emb(X_item)  # (B, T, E)
        emb_s = self.store_emb(X_store)  # (B, T, E)
        x = torch.cat([X_num, emb_i, emb_s], dim=-1)  # (B, T, F+2E)
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # Lấy output cuối cùng: (B, hidden_size)
        return self.fc(out).squeeze()

In [51]:
# Cell 4: Đọc thông tin từ model_params.txt và tái tạo DataLoader
with open('model_params.txt', 'r') as f:
    lines = f.readlines()
    num_items = int(lines[0].split(': ')[1])
    num_stores = int(lines[1].split(': ')[1])
    feature_cols = eval(lines[2].split(': ')[1])

# Tái tạo train_loader và val_loader
df = pd.read_csv('processed_data.csv')
split = int(len(df) * 0.8)
df_train, df_val = df.iloc[:split], df.iloc[split:]
train_ds = M5MultiDataset(df_train, feature_cols, window_size=28)
val_ds = M5MultiDataset(df_val, feature_cols, window_size=28)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)
print('Recreated train_loader and val_loader')

Recreated train_loader and val_loader


C:\Users\Ho Hau\AppData\Local\Temp\ipykernel_24320\1273609222.py:9: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('processed_data.csv')


In [52]:
# Cell 5: Khởi tạo mô hình, hàm mất mát, và optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMEmbForecast(
    n_items=num_items,
    n_stores=num_stores,
    embed_dim=16,
    num_feats=len(feature_cols),
    hidden_size=128
).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scaler = GradScaler()

C:\Users\Ho Hau\AppData\Local\Temp\ipykernel_24320\3332628159.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [53]:
print(f"CUDA available: {torch.cuda.is_available()}")

CUDA available: True


In [54]:
scaler = GradScaler()

def train_model(model, train_loader, val_loader, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for (X_num, X_item, X_store), y in train_loader:
            Xn, Xi, Xs, y = X_num.to(device), X_item.to(device), X_store.to(device), y.to(device)
            optimizer.zero_grad()
            with autocast(device_type='cuda'):  # Dùng đúng cú pháp mới
                preds = model(Xn, Xi, Xs)
                loss = criterion(preds, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        predictions, actuals = [], []
        with torch.no_grad():
            for (Xn, Xi, Xs), y in val_loader:
                Xn, Xi, Xs, y = Xn.to(device), Xi.to(device), Xs.to(device), y.to(device)
                with autocast(device_type='cuda'):
                    preds = model(Xn, Xi, Xs)
                    loss = criterion(preds, y)
                val_loss += loss.item()
                predictions.extend(preds.cpu().numpy())
                actuals.extend(y.cpu().numpy())
        
        # Đánh giá
        rmse = np.sqrt(mean_squared_error(actuals, predictions))
        mae = mean_absolute_error(actuals, predictions)
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {total_loss/len(train_loader):.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

C:\Users\Ho Hau\AppData\Local\Temp\ipykernel_24320\2433662127.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [55]:
# Cell 7: Huấn luyện mô hình
train_model(model, train_loader, val_loader, num_epochs=20, device=device)

Epoch 1/20 - Train Loss: 0.0055, Val Loss: 0.0023, RMSE: 0.0479, MAE: 0.0356
Epoch 2/20 - Train Loss: 0.0038, Val Loss: 0.0019, RMSE: 0.0440, MAE: 0.0317
Epoch 3/20 - Train Loss: 0.0033, Val Loss: 0.0017, RMSE: 0.0410, MAE: 0.0297
Epoch 4/20 - Train Loss: 0.0030, Val Loss: 0.0018, RMSE: 0.0424, MAE: 0.0298
Epoch 5/20 - Train Loss: 0.0029, Val Loss: 0.0016, RMSE: 0.0405, MAE: 0.0286
Epoch 6/20 - Train Loss: 0.0029, Val Loss: 0.0014, RMSE: 0.0380, MAE: 0.0277
Epoch 7/20 - Train Loss: 0.0029, Val Loss: 0.0014, RMSE: 0.0372, MAE: 0.0265
Epoch 8/20 - Train Loss: 0.0027, Val Loss: 0.0015, RMSE: 0.0389, MAE: 0.0271
Epoch 9/20 - Train Loss: 0.0027, Val Loss: 0.0016, RMSE: 0.0396, MAE: 0.0291
Epoch 10/20 - Train Loss: 0.0027, Val Loss: 0.0015, RMSE: 0.0389, MAE: 0.0270
Epoch 11/20 - Train Loss: 0.0027, Val Loss: 0.0014, RMSE: 0.0374, MAE: 0.0264
Epoch 12/20 - Train Loss: 0.0026, Val Loss: 0.0016, RMSE: 0.0395, MAE: 0.0283
Epoch 13/20 - Train Loss: 0.0026, Val Loss: 0.0013, RMSE: 0.0361, MAE: 0.

In [56]:
# Cell 8: Lưu mô hình đã huấn luyện
torch.save(model.state_dict(), 'lstm_forecast.pth')
print("Model saved to 'lstm_forecast.pth'")

Model saved to 'lstm_forecast.pth'
